In [32]:
# bibliotecas usadas
from imap_tools import MailBox
from bs4 import BeautifulSoup
import pandas as pd

In [33]:
#----------------------------------------------------------------------------------------#
#Função: acesso
#Parâmetros: 1) email: endereço de email ao qual pegaremos as mensagens
#            2) senha: senha do endereço de email acima 
#Retorna: objeto tipo "BaseMailBox.fetch" com acesso à caixa de entrada 
#-----------------------------------------------------------------------------------------#

def acesso(email, senha):
    #conexão
    username = email
    password = senha
    
    #Comando para a liberação do acesso à emails "@outlook"
    meu_email = MailBox('imap-mail.outlook.com').login(username, password)

    #cria um objeto gerador com os id's dos email
    lista_emails = meu_email.fetch("SEEN") #Possível de modificar para "UNSEEN: Emails não lidos, "ALL": lidos e não lidos, "SEEN": lidos
    
    return lista_emails

In [34]:
#----------------------------------------------------------------------------------------#
#Função: web_scraping
#Parâmetros: 1) lista_Email: objeto gerado que permite acessar email
#            2) quantidade: Quantidade de emails que deseja pegar 
#Retorna: Email desmembrado em: texto, assunto, para quem foi enviado, data de recebimento
#-----------------------------------------------------------------------------------------#

def web_scraping(lista_emails, quantidade):
    # gera listas com as informações (resposta, assunto, from e info_data)
    cont=0
    mail = []
    texto = []
    data = []
    subject = []
    para = []
    
    #Percorre o acesso do tipo "BaseMailBox.fetch", juntamente com a biblioteca Beatifulsoup para conseguir os dados adequados
    for email in lista_emails:
        p = email.html #Pega a página do email com todas as informações contidas nela
        soup = BeautifulSoup(p) #Instancia Beatifulsoup
        p = soup.get_text(strip=False) #Pega o texto
        p = p.replace('\n', ' ') #Tratamento do formato html 
        p = p.replace('\r', ' ') #Tratamento do formato html 
        p.strip()
        texto.append(p) #Armazena conteúdo principal (texto) do email 
        subject.append(email.subject) #Armazena assunto do email
        para.append(email.from_) #Armazena para qual email foi mandada a mensagem
        data.append(email.date) #Armazena data do recebimento do email
        cont+=1
        if cont == quantidade:
            break

    return texto, subject, para, data

In [35]:
#----------------------------------------------------------------------------------------#
#Função: tratamento
#Parâmetros: 1) texto: gerado da função anterior
#            2) subject: gerado da função anterior
#            2) para: gerado da função anterior 
#            2) data: gerado da função anterior
#Retorna: CSV no modelo adequado à próxima etapa de clusterização
#-----------------------------------------------------------------------------------------#

def tratamento(texto, subject, para, data):
    
    # cria-se um dataFrame vazio
    pp = pd.DataFrame()

    # atribui as informações no dataframe
    pp['EMAIL'] = para
    pp['RESPOSTA'] = texto
    pp['DATA'] = data
    pp['ASSUNTO'] = subject

    #Seleciona coluna de Respostas
    p = pp['RESPOSTA']
    #Colocando em string
    p = [str(x) for x in p]
    #Coloca as palavras em diminutivo
    p = [x.lower() for x in p]
    
    #Seleciona coluna de Assunto
    a = pp['ASSUNTO']
    #Colocando em string
    a = [str(x) for x in a]
    #Coloca as palavras em diminutivo
    a = [x.lower() for x in a]
    
    #Pegando das respostas, as respostas até o primeiro ponto ou ate o primeiro "de:" e armazenando em coluna 'resp'    
    resp = []
    for i in range(len(pp)):
        if 're: ' in a[i]:
            pos1 = p[i].find('de: ')
            if pos1 == -1:
                pos1 = p[i].find('.')
                if pos1 > 0 and pos1 < 35:
                    aux = p[i][pos1+1:]
                    pos1 = pos1+aux.find('.')
                    if pos1 == -1:
                        pos1 = len(aux)-1
                elif pos1 == -1:
                    pos1=len(p[i])-1
            aux = p[i][:pos1]
            resp.append(aux)
        else:
            pos1 = p[i].find('.')
            if pos1 > 0 and pos1 < 35:
                aux = p[i][pos1+1:]
                pos1 = pos1+aux.find('.')
                if pos1 == -1:
                    pos1 = len(aux)-1
            elif pos1 == -1:
                pos1 = len(p[i])-1
            aux = p[i][:pos1+1]
            resp.append(aux)
            
    pp['RESPOSTA'] = resp
    
    #Arquivos onde ocorre a substituição de palvras importadas que continham erros padronizados, para a palavra de seu significado
    Certo = open("LOCAL_DO_ARQUIVO\Certo.txt", "r", encoding='utf-8')
    Errado = open("LOCAL_DO_ARQUIVO\Errado.txt", "r", encoding='utf-8')
    
    #Armazena em lista e retira "\n" que o "readlines" atribui ao ler o txt
    aux = Certo.readlines()
    certo = []
    for i in range(len(aux)):
        certo.append(aux[i].rstrip("\n'"))
        
    #Armazena em lista e retira "\n" que o "readlines" atribui ao ler o txt
    aux2 = Errado.readlines()    
    errado = []
    for i in range(len(aux2)):
        errado.append(aux2[i].rstrip("\n'"))
        
    #Realiza troca das palvras com erros para palavras corrigidas
    for i in range(len(certo)):
        pp['RESPOSTA'].replace(regex=True, inplace=True, to_replace=errado[i], value=certo[i])

    #Para a clusterização, não necessitaremos do "Assunto"
    pp = pp.drop(columns='ASSUNTO')
    
    # exporta o dataframe com as modificações
    return pp.to_csv("LOCAL_DO_ARQUIVO\EMAIL.csv", index=False)